In [99]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf

# Supongamos que 'data' es el DataFrame cargado con las columnas descritas
data=pd.read_csv('../../Data/Limpio/dataset_limpio_parte1.csv')
data1=pd.read_csv('../../Data/Limpio/dataset_limpio_parte2.csv')
data2=pd.read_csv('../../Data/Limpio/dataset_limpio_parte3.csv')
data_unida=pd.concat([data,data1,data2]) # Ajusta el nombre del archivo según corresponda
data_unida.head()

/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_11460/2895795965.py:12: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('../../Data/Limpio/dataset_limpio_parte1.csv')
/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_11460/2895795965.py:13: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data1=pd.read_csv('../../Data/Limpio/dataset_limpio_parte2.csv')
/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_11460/2895795965.py:14: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data2=pd.read_csv('../../Data/Limpio/dataset_limpio_parte3.csv')


,cod_persona,mes,pais,sexo,edad,fecha1,xti_empleado,xti_nuevo_cliente,num_antiguedad,xti_rel,...,ind_prod16,ind_prod17,ind_prod18,ind_prod19,ind_prod20,ind_prod21,ind_prod22,ind_prod23,ind_prod24,ind_prod25
0,5514,2016-04-28,ES,H,44,2016-04-29,N,1.0,0,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0
1,5541,2016-04-28,ES,H,60,2016-04-29,N,1.0,0,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0
2,5631,2016-04-28,ES,V,53,2016-04-28,N,1.0,0,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0
3,5656,2016-04-28,ES,H,47,2016-04-28,N,1.0,0,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0
4,5738,2016-04-28,ES,V,48,2016-04-28,N,1.0,0,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0


In [56]:
data_unida.loc[data_unida['edad']==' NA','edad']='-1'
data_unida.loc[data_unida['num_antiguedad']=='     NA','num_antiguedad']='-1'

In [57]:
data=data_unida[~data_unida['fecha1'].isna()]
data['edad']=data['edad'].astype(float)
data['num_antiguedad']=data['num_antiguedad'].astype(float)
data['fecha1'] = pd.to_datetime(data['fecha1'])
data['mes'] = pd.to_datetime(data['mes'])

/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_11460/3402631540.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['edad']=data['edad'].astype(float)
/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_11460/3402631540.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['num_antiguedad']=data['num_antiguedad'].astype(float)
/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_11460/3402631540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [60]:
columnas_cat=data.select_dtypes(include='O').columns

onehot=OneHotEncoder(sparse_output=False)
data_onehot=onehot.fit_transform(data[columnas_cat])
data_onehot=pd.DataFrame(data_onehot,columns=onehot.get_feature_names_out(columnas_cat))
df_onehot = data_onehot.reset_index(drop=True)
data = data.reset_index(drop=True)

data=pd.concat([data.drop(columnas_cat,axis=1),df_onehot],axis=1)
data.head()

,cod_persona,mes,edad,fecha1,xti_nuevo_cliente,num_antiguedad,xti_rel,tip_dom,cod_provincia,xti_actividad_cliente,...,des_canal_KHO,des_canal_KHP,des_canal_KHQ,des_canal_RED,xti_extra_N,xti_extra_S,id_segmento_01 - TOP,id_segmento_02 - PARTICULARES,id_segmento_03 - UNIVERSITARIO,id_segmento_04-NO TIENE
0,5514,2016-04-28,44.0,2016-04-29,1.0,0.0,1.0,1.0,8.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,5541,2016-04-28,60.0,2016-04-29,1.0,0.0,1.0,1.0,41.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,5631,2016-04-28,53.0,2016-04-28,1.0,0.0,1.0,1.0,15.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,5656,2016-04-28,47.0,2016-04-28,1.0,0.0,1.0,1.0,46.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,5738,2016-04-28,48.0,2016-04-28,1.0,0.0,1.0,1.0,28.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [63]:
data = data.sort_values(by=['cod_persona', 'fecha1'])
cods_persona=data['cod_persona']
data.drop('mes',axis=1,inplace=True)

In [65]:
import numpy as np

def create_sequences(df, cod_persona_col, feature_cols, target_cols, sequence_length):
    sequences = []
    targets = []
    
    personas = df[cod_persona_col].unique()
    for persona in personas:
        persona_df = df[df[cod_persona_col] == persona]
        persona_df = persona_df.reset_index(drop=True)
        
        for i in range(len(persona_df) - sequence_length):
            sequence = persona_df.loc[i:i+sequence_length-1, feature_cols].values
            target = persona_df.loc[i+sequence_length, target_cols].values
            sequences.append(sequence)
            targets.append(target)
    
    return np.array(sequences), np.array(targets)

# Definir las columnas de características y de objetivos
feature_cols = [col for col in data.columns if not col.startswith('ind_') and col not in ['cod_persona', 'fecha1']]
target_cols = [col for col in data.columns if col.startswith('ind_')]

# Longitud de la secuencia
sequence_length = 4  # Por ejemplo, secuencias de 5 días

# Crear las secuencias
X, y = create_sequences(data, 'cod_persona', feature_cols, target_cols, sequence_length)


In [66]:
len(y[0])

25

In [67]:
X

array([[[ 29.,   1.,   0., ...,   0.,   0.,   1.],
        [ 29.,   1.,   0., ...,   1.,   0.,   0.],
        [ 30.,   1.,   2., ...,   0.,   1.,   0.],
        [ 30.,   1.,   3., ...,   0.,   1.,   0.]],

       [[ 34.,   1.,   0., ...,   0.,   0.,   1.],
        [ 33.,   1.,   0., ...,   1.,   0.,   0.],
        [ 33.,   1.,   2., ...,   1.,   0.,   0.],
        [ 33.,   1.,   3., ...,   1.,   0.,   0.]],

       [[ 52.,   1.,   0., ...,   0.,   0.,   1.],
        [ 52.,   1.,   0., ...,   1.,   0.,   0.],
        [ 52.,   1.,   2., ...,   1.,   0.,   0.],
        [ 52.,   1.,   3., ...,   1.,   0.,   0.]],

       ...,

       [[ 56.,   0., 248., ...,   0.,   0.,   0.],
        [ 56.,   0., 249., ...,   0.,   0.,   0.],
        [ 56.,   0., 250., ...,   0.,   0.,   0.],
        [ 56.,   0., 251., ...,   0.,   0.,   0.]],

       [[ 56.,   0., 249., ...,   0.,   0.,   0.],
        [ 56.,   0., 250., ...,   0.,   0.,   0.],
        [ 56.,   0., 251., ...,   0.,   0.,   0.],
        [ 

In [69]:
cutoff = int(len(X) * 0.8)

# Dividir los datos en entrenamiento y prueba manteniendo la temporalidad
X_train, X_test = X[:cutoff], X[cutoff:]
y_train, y_test = y[:cutoff], y[cutoff:]

X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.float32)

In [84]:

def crear_modelo_denso(input_shape,output_shape, num_layers=3, units_per_layer=64, dropout_rate=0.5):
    model = Sequential()
    model.add(tf.keras.layers.LSTM(units = 64
               , return_sequences=True
               , dropout=0.3
               , input_shape = input_shape))
    model.add(tf.keras.layers.LSTM(units=64,return_sequences=False
               , dropout=0.3))
    
    for _ in range(num_layers - 1):
        model.add(Dense(units_per_layer, activation='relu'))
        model.add(Dropout(dropout_rate))
        
    model.add(Dense(output_shape, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [111]:
model=crear_modelo_denso(X_train_tensor.shape[-2:],y_test_tensor.shape[1],12,81)
model.summary()

/Users/mariolamas/anaconda3/envs/entorno_prueba/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_8 (LSTM)                   │ (None, 4, 64)          │        68,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 81)             │         5,265 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 81)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 81)             │         6,642 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 81)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 81)             │         6,642 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 81)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 81)             │         6,642 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 81)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 81)             │         6,642 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 81)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 81)             │         6,642 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 81)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 81)             │         6,642 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 81)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 81)             │         6,642 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 81)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 81)             │         6,642 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 81)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 81)             │         6,642 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_28 (Dropout)            │ (None, 81)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 81)             │         6,642 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_29 (Dropout)            │ (None, 81)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 25)             │         2,05

 Total params: 175,623 (686.03 KB)

 Trainable params: 175,623 (686.03 KB)

 Non-trainable params: 0 (0.00 B)

In [112]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=1),  # Parada temprana si la pérdida en validación deja de disminuir
    ModelCheckpoint('modelo_epoch_{epoch:02d}.keras', save_freq='epoch', verbose=1)  # Guardar el modelo en cada época
]

In [113]:
history=model.fit(X_train_tensor,y_train_tensor,epochs=100,batch_size=16,validation_data=(X_test_tensor,y_test_tensor),callbacks=callbacks)

Epoch 1/100


22447/22450 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6986 - loss: 0.1332
Epoch 1: saving model to modelo_epoch_01.keras
22450/22450 ━━━━━━━━━━━━━━━━━━━━ 186s 8ms/step - accuracy: 0.6986 - loss: 0.1332 - val_accuracy: 0.5573 - val_loss: 0.2516
Epoch 2/100
 4001/22450 ━━━━━━━━━━━━━━━━━━━━ 2:40 9ms/step - accuracy: 0.7129 - loss: 0.1191

KeyboardInterrupt: 

In [109]:
#Prueba de prediccion sobre 1 ejemplo
model.predict(tf.expand_dims(X_test_tensor[0],axis=0)) 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step


array([[2.7724370e-06, 3.7985856e-08, 6.1928725e-01, 3.1681509e-06,
        4.2065173e-02, 2.4940720e-02, 1.2245566e-02, 1.0531313e-01,
        3.6595836e-02, 5.7378518e-05, 2.0331277e-04, 1.5583991e-02,
        5.3284641e-02, 4.6213842e-03, 8.8086847e-04, 1.2277312e-03,
        1.6586343e-03, 3.0047741e-02, 1.4659212e-02, 5.6546717e-03,
        2.4631114e-03, 2.8762029e-02, 2.9868940e-02, 7.1284823e-02,
        1.8808149e-02]], dtype=float32)

In [110]:
y_train_tensor[0]

<tf.Tensor: shape=(25,), dtype=float32, numpy=
array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 0.], dtype=float32)>

In [ ]:
loss, accuracy = model.evaluate(X_test_tensor, y_test_tensor)
print(f"Loss: {loss}, Accuracy: {accuracy}")

# Definir la ruta donde se guardará el modelo
model_path = './mi_modelo.h5'

# Guardar el modelo
model.save(model_path)
print(f'Modelo guardado en: {model_path}')

# Cargar el modelo
modelo_cargado = load_model(model_path)

# Verificar que el modelo ha sido cargado correctamente
modelo_cargado.summary()

2807/2807 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.5582 - loss: 0.2248


Loss: 0.24591977894306183, Accuracy: 0.5573100447654724
Modelo guardado en: ./mi_modelo.h5


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 4, 64)          │        68,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 81)             │         5,265 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 81)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 81)             │         6,642 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 81)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 81)             │         6,642 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 81)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 81)             │         6,642 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 81)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 81)             │         6,642 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 81)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 25)             │         2,050 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 135,773 (530.37 KB)

 Trainable params: 135,771 (530.36 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [ ]:
import pickle

# Definir la ruta donde se guardará el historial
history_path = './history.pkl'

# Guardar el historial
with open(history_path, 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
# Cargar el historial
with open(history_path, 'rb') as file_pi:
    history = pickle.load(file_pi)

print('Historial cargado correctamente')